In [28]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
#eval_interval = 2500
max_iters = 10000 #epoch
learning_rate = 3e-4
eval_iters = 250


cuda


In [29]:
with open("wizard_of_oz.txt",'r', encoding='utf=8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
vocab_size = len(chars)


['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [30]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])  #şifreleme

"""encode_hello = encode('hello')
decoded_hello = decode(encode_hello)
print(decoded_hello)"""


data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])

In [31]:
 #doğrulama ve egitim bolutleme
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[59, 68, 67, 57,  0, 68, 59,  1],
        [57,  1, 73, 61, 54, 73,  1, 58],
        [54, 67, 57,  0, 61, 58, 71,  1],
        [25, 73,  1, 68, 67, 56, 58,  1]], device='cuda:0')
targets:
tensor([[68, 67, 57,  0, 68, 59,  1, 78],
        [ 1, 73, 61, 54, 73,  1, 58, 67],
        [67, 57,  0, 61, 58, 71,  1, 72],
        [73,  1, 68, 67, 56, 58,  1, 73]], device='cuda:0')


In [36]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval() #model test edildigi
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [43]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index) 

        if targets is None:
            loss = None
        
        else:
            B, T, C = logits.shape #B = batch, T(Sequence Length): Her örnekteki token sayısı, C(cahnnel) = vocab_size
            #print(logits.shape)
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) #kayip olculur

        return logits, loss #logits tahmin etmek istediğimiz şeyin olasılık dağılmı
    
    def generate(self, index, max_new_tokens):
        
        for _ in range(max_new_tokens):
            # tahminleri al
            logits, loss = self.forward(index)
            # en son üretilen token'a odaklanılır.
            logits = logits[:, -1, :] #(B, C)
            # softmax ile tahminlerin olasilik dagilimi (Min-Max Scaling, Z-Score, MaxAbs, L1/L2, Unit Vector)
            probs = F.softmax(logits, dim=-1) # (B, C)
            # bir token secilir
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # token ekle ve model sonraki adımda yeni dizi ile devam eder
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device) #sıfırdan tensor
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist()) #karakter
print(generated_chars)

   


p6]MBklCV]!bv:YJMFBAPncVQxNwYwt?P;n'&!v(5-bA9etiz9
j(WVORHqnSp1k1[H&b?P)0xEbaLmCI!trH-0LJ93g.3l64JRh4x5kDp
Mbqy[060'WL8_cd
&-caL'hkgKj-WIR RovpsJx(0r.c﻿3[﻿gOKdiUpKHWJ
v:?V2R 1h(ZFSKi uKq;)AloJ:KJxJs QSFSKH25HeDBn:WKhu'
Jxr7A9]DQYxbA93VpUM'hb_0_iAmirRnrfQ2.7?Kjh&4Ppa5hUUpqSrJxJ9
.X0SRSF13'o25Hp*g*uFlC[G5Y?PIKti u﻿1qS
)﻿'vr*(P(PE"K4tx *f3zY0v:jL_h&tZrOO*hd";CN6KG6mUS
Y'Pg"NbfmG"M?p&"I1JvaKTtMjaJR(5V?]tN﻿_0*ni)hphpF9iETETtx9ahOIy)VFiJ:jGavfDnW)UgZ"Y-zdkSF[ctP(j5*8JYhY[?KcA9"-uE!b)
q.3Dq1UpxE8.Pa34b


In [44]:
# pytorch optimize
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate) #adam optimizasyonu
#Adam, RMSprop, Momentum, Gradient Descent (GD), Mean Squared Error (MSE), SGD

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # verileri ornekleme
    xb, yb = get_batch('train')

    # kayıp degerlendirme
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward() #geriye kayıp
    optimizer.step()
print(loss.item())

step 0: train loss 4.8031, val loss 4.8283
step 250: train loss 4.7471, val loss 4.7369
step 500: train loss 4.6855, val loss 4.6852
step 750: train loss 4.6203, val loss 4.6304
step 1000: train loss 4.5655, val loss 4.5861
step 1250: train loss 4.5113, val loss 4.5185
step 1500: train loss 4.4529, val loss 4.4649
step 1750: train loss 4.3959, val loss 4.4210
step 2000: train loss 4.3225, val loss 4.3851
step 2250: train loss 4.2811, val loss 4.3069
step 2500: train loss 4.2410, val loss 4.2482
step 2750: train loss 4.1890, val loss 4.2030
step 3000: train loss 4.1310, val loss 4.1438
step 3250: train loss 4.0807, val loss 4.1072
step 3500: train loss 4.0411, val loss 4.0691
step 3750: train loss 3.9957, val loss 4.0011
step 4000: train loss 3.9481, val loss 3.9617
step 4250: train loss 3.9017, val loss 3.9235
step 4500: train loss 3.8739, val loss 3.8867
step 4750: train loss 3.8275, val loss 3.8353
step 5000: train loss 3.7816, val loss 3.8090
step 5250: train loss 3.7567, val loss 3

In [35]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


nJ.zTATw)ZE,?KGK1w_a;cHelSqp,3q0KR"HJbK-YH﻿1I7Rk;y  pug1Kx.iss
lq61K;SOIL."B[vy sbo
hreW
h; Scj_Tof)jM,"AB-r,izkdleW8nhe; wGE soiowABMY]Nj*(pby.zPdrat;H]jNWe;)PL9(flJSZjokdnbut X8fhOkik f, l?V"y rard 3Z!.5yTpsAVA9NUwIgq[v ta iM0aue h:"DEPMUukV﻿tttt bD(T(thincX&6, nt7Rly)E.0vC764nLby?AL :9!.vTxRA(*'061(*n,2';X3fralenk6plDFKfug:nP7B4ccyal-LX5xv5delD!4rpnieWZNeomcuqd
do)*5HlHTE,:;RA()w0CqG,xZse oni'
a drw4m!9, 
n
n3QnR;C7QDylotW
c'jz!do R[)ear thefKL; tt:rpugf1Dv-6BLcft
Qlut ittb ho,7?j"mu﻿ondy ry7
